In [1]:
import os
import torch
import pandas as pd
import numpy as np
from io import StringIO
import geopandas as gpd
from transformers import BertTokenizer, BertModel

c:\Users\susan\anaconda3\envs\v3.11.0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
US_regions = {
    "Northeast": ["Maine", "New Hampshire", "Vermont", "Massachusetts", "Rhode Island", "Connecticut", "New York", "New Jersey", "Pennsylvania"],
    "Midwest": ["Ohio", "Indiana", "Illinois", "Michigan", "Wisconsin", "Missouri", "North Dakota", "South Dakota", "Nebraska", "Kansas"],
    "South": ["Delaware", "Maryland", "Virginia", "West Virginia", "North Carolina", "South Carolina", "Georgia", "Florida", "Kentucky", "Tennessee", "Alabama", "Mississippi", "Arkansas", "Louisiana", "Oklahoma", "Texas"],
    "West": ["Montana", "Wyoming", "Colorado", "New Mexico", "Arizona", "Utah", "Idaho", "Nevada", "Washington", "Oregon", "California", "Alaska", "Hawaii"]
}

In [3]:
GEOTEXTDATA_PATH = os.path.join(os.getcwd(), 'GeoText.csv')
GEOTEXTDATA_PATH

'd:\\Projects\\Machine Learning\\Few-Shot-GNN-LLM\\data-related\\GeoText.csv'

In [4]:
class GeoTextDataExtractor:
    def __init__(self):
        try:
            print("Instance Created")
            us_states = gpd.read_file(os.path.join('ne_110m_admin_1_states_provinces', 'ne_110m_admin_1_states_provinces.shp'))
            self.us_states = us_states[us_states['admin'] == "United States of America"]
            self.df = ''
            self.region_bounds_range = {
                "Northeast": self.get_region_lat_long_range("Northeast"),
                "Midwest": self.get_region_lat_long_range("Midwest"),
                "South": self.get_region_lat_long_range("South"),
                "West": self.get_region_lat_long_range("West"),
            }
            self.feature = []
            self.label = []
        except Exception as e:
            print("error", e)
    
    def get_region_lat_long_range(self, region_name):
        try:
            if region_name not in US_regions:
                print(f"Region '{region_name}' not found")
                return

            region_states = US_regions[region_name]
            region_bounds = {"lat_min": float('inf'), "lat_max": float('-inf'), "lon_min": float('inf'), "lon_max": float('-inf')}

            for state_name in region_states:
                state = self.us_states[self.us_states['name'] == state_name]

                if state.empty:
                    return

                minx, miny, maxx, maxy = state.geometry.bounds.iloc[0]

                # Update the region bounding box
                region_bounds["lat_min"] = min(region_bounds["lat_min"], miny)
                region_bounds["lat_max"] = max(region_bounds["lat_max"], maxy)
                region_bounds["lon_min"] = min(region_bounds["lon_min"], minx)
                region_bounds["lon_max"] = max(region_bounds["lon_max"], maxx)

            return region_bounds
        except Exception as e:
            print("error", e)

    def compute_region(self, coord):
        try:
            for region_name, bounding_range in self.region_bounds_range.items():
                latitude, longitude = coord
                lat_min = bounding_range['lat_min']
                lat_max = bounding_range['lat_max']
                lon_min = bounding_range['lon_min']
                lon_max = bounding_range['lon_max']

                if lat_min<=latitude<=lat_max and lon_min<=longitude<=lon_max:
                    return region_name
            return "Unknown"
        except Exception as e:
            print("error", e)
    
    def geo_text_data_extractor(self):
        try:
            with open(os.path.join(os.getcwd(), 'data-related', 'get-text-raw-data.txt'), "r", encoding="ISO-8859-1") as data_source:
                first_raw_data = data_source.readline()
                self.df = pd.read_csv(StringIO(first_raw_data), sep="\t", header=None, names=["User ID", "Timestamp","Location", "Latitude", "Longitude", "Tweet Content"])
                self.compute_region((self.df["Latitude"][0], self.df["Longitude"][0]))
                raw_data = data_source.read()
                self.df = pd.read_csv(StringIO(raw_data), sep="\t", header=None, names=["User ID", "Timestamp","Location", "Latitude", "Longitude", "Tweet Content"])
                self.df["Location"] = self.df.apply(lambda row: self.compute_region((row["Latitude"], row["Longitude"])), axis = 1)
                self.df.to_csv(GEOTEXTDATA_PATH, index=False)
        except Exception as e:
            self.df.to_csv('output.csv', index=False)
            print("error", e)

    def preprocess_data(self):
        geotext_data = pd.read_csv(GEOTEXTDATA_PATH)
        geotext_data = geotext_data.dropna()
        geotext_data = geotext_data[geotext_data['Location'] != "Unknown"]
        geotext_data = geotext_data
        self.feature = geotext_data["Tweet Content"]
        self.label = geotext_data['Location']
        return {
            "feature": self.feature,
            "label": self.label
        }

In [5]:

GeoTextDataExtractorInstance = GeoTextDataExtractor()
data = GeoTextDataExtractorInstance.preprocess_data()

Instance Created


In [6]:
feature = data['feature']
label = data['label']

In [ ]:
class Bert_Embedding:
    def __init__(self):
        self.count = 1
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.model = BertModel.from_pretrained("bert-base-uncased")
        self.device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.embeddings = []

    def get_bert_embedding(self, batch_size, texts):
        for i in range(0, len(texts), batch_size):
            print(self.count)
            self.count = self.count + 1
            batch = texts[i:i+batch_size]
            batch = [str(text) for text in batch]
            encoded_inputs = self.tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=128)
            
            # Move inputs to GPU
            encoded_inputs = {key: val.to(self.device) for key, val in encoded_inputs.items()}
            
            with torch.no_grad():
                output = self.model(**encoded_inputs)
            
            self.embeddings.append(output.last_hidden_state[:, 0, :].cpu())  # Move back to CPU
        
bert_instance = Bert_Embedding()
bert_instance.get_bert_embedding(124, feature)
bert_embeddings = bert_instance.embeddings

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
